In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [38]:
df = pd.read_csv('cars.csv', encoding= 'latin1')
df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'cars.csv'

In [34]:
df1 = df.apply(lambda x: x.fillna(x.value_counts().index[0]))
df1.isnull().sum()

YEAR                0
MONTH               0
SUPPLIER            0
ITEM CODE           0
ITEM DESCRIPTION    0
ITEM TYPE           0
RETAIL SALES        0
RETAIL TRANSFERS    0
WAREHOUSE SALES     0
dtype: int64

In [35]:
df1.groupby('ITEM TYPE')['ITEM TYPE'].agg('count')

ITEM TYPE
BEER             42413
DUNNAGE             95
KEGS             10146
LIQUOR           64910
NON-ALCOHOL       1908
REF                127
STR_SUPPLIES       405
WINE            187641
Name: ITEM TYPE, dtype: int64